<a href="https://colab.research.google.com/github/CallMeL/OML-hw/blob/master/newtons_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise sheet 5

**Please turn in your exercises by Monday, December 9th.**

## Task 1: Newton's step

Prove the following corollary.

**Corollary**

Let $f\colon \mathbb{R}^d \to \mathbb{R}$ be twice differentiable, $f \in \mathcal{S}_{\mu,L}$ and $\|\nabla^2 f(x) - \nabla^2 f(y)\| \leq M \|x - y\|$.

If for some $t_0$ we have
$$
\|x_{t_0} - x^\star\| \leq \frac{\mu}{M}
$$
then $\forall t > t_0$ we have
$$
\|x_t - x^\star\|\leq \frac{\mu}{2M} \left(\frac{1}{2}\right)^{2^{t - t_0}}
$$
and
$$
f(x_t) - f(x^\star) \leq \frac{L\mu}{2M}\left(\frac{1}{2}\right)^{2^{t - t_0}}
$$
**Proof**
$$
...
$$


## Task 2: Affine invariance

Prove the following lemma.

**Lemma**

Let $f\colon \mathbb{R}^d \to \mathbb{R}$ be twice differentiable, $A \in \mathbb{R}^{d \times d}$ an invertible matrix, $b \in \mathbb{R}^d$.
Let $g\colon \mathbb{R}^d \to \mathbb{R}^d$ be the (bijective) function
$$
g(x) := Ax + b.
$$
Finally, for a twice differentiable function $h\colon \mathbb{R}^d \to \mathbb{R}$, let $N_h\colon \mathbb{R}^d \to \mathbb{R}^d$ denote the Newton step for $h$, i.e.
$$
N_h(x) := x - \nabla^2 h(x)^{-1} \nabla h(x),
$$
whenever this is definde. Then
$$
N_{f \circ g} := g^{-1} \circ N_f \circ g.
$$

**Proof**
$$
...
$$

## Task 3: Non-quadratic convergence example

Let $\delta > 0$ be a real number. Find an example of a convex function $f \colon \mathbb{R} \to \mathbb{R}$ such that Newton's method satisfies
$$
|x_{t+1} - x^\star| \geq (1 - \delta) |x_t + x^\star|
$$
for all $x_t \neq x^\star$. For this function, Newton's method will not have a local quadratic convergence phase.

## Task 4: Babylonian method

Prove that you need $O(\log R)$ iterations to get $x_t - \sqrt{R} < 1/2$ using the Babylonian method.

## Task 5: Positive definite bound

Prove that $\|Mx\|_2 \leq L\|x\|_2$ for all positive semidefinite $M$ with $L\cdot \mathbb{I} \succeq M$.

## Utilities

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [ ]:
def contour_map(f, xb=(-1,1), yb=(-1,1), ax=None):
    if ax is None:
        ax = plt
    (nx, ny) = (45, 45)
    x = np.linspace(*xb, nx)
    y = np.linspace(*yb, ny)
    xv, yv = np.meshgrid(x, y)
    X = np.block([ [xv.reshape(1, -1)], [yv.reshape(1, -1)] ]).T
    zv = np.fromiter((f(x) for x in X), dtype=np.double)
    zv = zv.reshape(nx,ny)
    ax.contour(xv, yv, zv, 15)

def surface_plot(f, xb=(-1,1), yb=(-1,1)):
    (nx, ny) = (45, 45)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x = np.linspace(*xb, nx)
    y = np.linspace(*yb, ny)
    xv, yv = np.meshgrid(x, y)
    X = np.block([ [xv.reshape(1, -1)], [yv.reshape(1, -1)] ]).T
    zv = np.fromiter((f(x) for x in X), dtype=np.double)
    zv = zv.reshape(nx,ny)
    ax.plot_surface(xv, yv, zv, cmap=cm.coolwarm)
    return fig, ax

In [ ]:
def backtracking_line_search(x, d, f, g, alpha=0.3, beta=0.8):
    step_size = 1.
    while f(x + step_size * d) > f(x) + alpha * step_size * g(x).dot(d):
        step_size *= beta
    return step_size

def backtracking_line_search_nag(x, d, f, g, alpha=1., beta=0.8):
    t = 1
    while f(x + t*d) > f(x) + alpha * t * np.dot(g(x), d) + t/2*np.dot(d, d):
        t *= beta
    return t


In [ ]:
def gradient_descent_path(x0, f, g, max_iter=100):
    xs = [x0]
    for _ in range(max_iter):
        step = backtracking_line_search(x0, -g(x0), f, g)
        x0 = x0 - step * g(x0)
        xs.append(x0)
    return xs

def heavy_ball_path(x0, f, g, max_iter=100):
    xs = [x0]
    x_old = x0
    for i in range(1, max_iter+1):
        d = -g(x0) + (i-1)/(i+1) *  (x0-x_old)
        x_old = x0
        t = backtracking_line_search_nag(x0, d, f, g)
        x0 = x0 + t * d
        xs.append(x0)
    return xs

def nag_path(x0, f, g, max_iter=100):
    xs = [x0]
    x_old = x0
    for i in range(1, max_iter+1, 1):
        y = x0 + (i-1)/(i+2) *  (x0-x_old)
        x_old = x0
        t = backtracking_line_search_nag(y, -g(y), f, g)
        x0 = y - t * g(y)
        xs.append(x0)
    return np.array(xs)

In [ ]:
EXA = np.array([[30., 15],[-20, 25]])/20

EXAMPLES = [
    (lambda x: x.T@EXA@x + 1, lambda x: (EXA+EXA.T)@x, lambda x: (EXA+EXA.T), 0.5 * np.ones(2), (-1.,1.), (-1.,1.)),
    (lambda x: (x[0]**2 + 30 * x[1]**2 + 4 * x[0]), lambda x: np.array([2 * x[0] + 4, 60 * x[1]]), lambda x: np.array([[2, 0],[0, 60]]), np.array([2.,3.]), (-2.5,2.5), (-1.5,3.5)),
    (lambda x: np.linalg.norm(np.sin(x*3))**2, lambda x: 6 * np.sin(x*3) * np.cos(x*3), lambda x: 18 * np.diag(2 * np.cos(3*x)**2 - 1), np.array([.2,.15]), (-.5, .5), (-.5,.5))
]

def run_examples_newtons_method():
    for (f, g, h, x0, xb, yb) in EXAMPLES:
        xs2 = newtons_method_path(x0, f, g, h)
        xs2 = np.array(xs2)

        xs = gradient_descent_path(x0, f, g)
        xs = np.array(xs)

        contour_map(f, xb=xb, yb=yb)
        plt.plot(xs2[:,0], xs2[:,1], '.--k', label='newtons method')
        plt.plot(xs[:,0], xs[:,1], '.--', color='gray', alpha=.5, label='gradient descent')
        plt.legend()
        plt.show()

## Task 6: Newton's method

Implement Newton's method using the above provided backtracking line search.
* `x0` is the initial point.
* `f` is the function you are trying to minimize.
* `g` is the gradient of `f`.
* `h` is the hessian of `f`.

Function `newtons_method_path` should return a list of vectors on the path to the minimum.

In [ ]:
def newtons_method_path(x0, f, g, h, max_iter=100):
    return ...

In [ ]:
# run this to get plots
run_examples_newtons_method()

## Task 7: Error plots

Complete the hessians of the following functions.
Then compare and plot the error over time for the four methods on the following tasks.
Implementations of the prior methods are given in the Utils section.

### Simple quadratic function

In [ ]:
def f(x):
    return x[0]**2 + 30 * x[1]**2 + 4 * x[0]
def g(x):
    return np.array([2 * x[0] + 4, 60 * x[1]])
def h(x):
    return ...

x0 = np.array([2.,3.])

x_star = np.array([-2.,0.])
...

### Linear regression

In [ ]:
from sklearn.datasets import make_regression

def f(w):
    return np.linalg.norm(X @ w - y) ** 2 / len(X)

def g(w):
    return 2 * X.T @ (X @ w - y) / len(X)

def h(w):
    return ...

X, y = make_regression(n_samples=1000, n_features=100, n_informative=40, random_state=0)
x0 = np.zeros(100)

x_star = np.linalg.lstsq(X, y, rcond=None)[0]
...

### Logistic regression


In [ ]:
from sklearn.datasets import make_classification
from scipy.optimize import minimize

def f(w):
    return np.log(1. + np.exp(-y * X.dot(w))).mean() + np.linalg.norm(w)**2

def g(w):
    sig = np.exp(-y * X.dot(w))
    return 2*w - X.T.dot(sig * y / (sig + 1.)) / X.shape[0]

def h(w):
    return ...

X, y = make_classification(1000, 80, n_informative=40,
#                               n_redundant=0,
                               n_clusters_per_class=2, flip_y=0.1, random_state=0)

x0 = np.zeros(80)
x_star = minimize(f, x0, jac=g).x
...